# Anna KaRNNa

In this notebook, I'll build a character-wise RNN trained on Anna Karenina, one of my all-time favorite books. It'll be able to generate new text based on the text from the book.

This network is based off of Andrej Karpathy's [post on RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) and [implementation in Torch](https://github.com/karpathy/char-rnn). Also, some information [here at r2rt](http://r2rt.com/recurrent-neural-networks-in-tensorflow-ii.html) and from [Sherjil Ozair](https://github.com/sherjilozair/char-rnn-tensorflow) on GitHub. Below is the general architecture of the character-wise RNN.

<img src="assets/charseq.jpeg" width="500">

In [1]:
import time
from collections import namedtuple

import numpy as np
import tensorflow as tf

First we'll load the text file and convert it into integers for our network to use.

In [2]:
with open('anna.txt', 'r') as f:
    text=f.read()
vocab = set(text)
vocab_to_int = {c: i for i, c in enumerate(vocab)}
int_to_vocab = dict(enumerate(vocab))
chars = np.array([vocab_to_int[c] for c in text], dtype=np.int32)

In [3]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

In [4]:
chars[:100]

array([30, 55, 76, 12, 66, 65, 40, 11, 78,  8,  8,  8, 28, 76, 12, 12, 13,
       11, 24, 76, 14, 57, 21, 57, 65, 46, 11, 76, 40, 65, 11, 76, 21, 21,
       11, 76, 21, 57, 56, 65, 43, 11, 65, 26, 65, 40, 13, 11, 38, 25, 55,
       76, 12, 12, 13, 11, 24, 76, 14, 57, 21, 13, 11, 57, 46, 11, 38, 25,
       55, 76, 12, 12, 13, 11, 57, 25, 11, 57, 66, 46, 11, 47, 77, 25,  8,
       77, 76, 13, 23,  8,  8, 58, 26, 65, 40, 13, 66, 55, 57, 25])

Now I need to split up the data into batches, and into training and validation sets. I should be making a test set here, but I'm not going to worry about that. My test will be if the network can generate new text.

Here I'll make both input and target arrays. The targets are the same as the inputs, except shifted one character over. I'll also drop the last bit of data so that I'll only have completely full batches.

The idea here is to make a 2D matrix where the number of rows is equal to the number of batches. Each row will be one long concatenated string from the character data. We'll split this data into a training set and validation set using the `split_frac` keyword. This will keep 90% of the batches in the training set, the other 10% in the validation set.

In [5]:
def split_data(chars, batch_size, num_steps, split_frac=0.9):
    """ 
    Split character data into training and validation sets, inputs and targets for each set.
    
    Arguments
    ---------
    chars: character array
    batch_size: Size of examples in each of batch
    num_steps: Number of sequence steps to keep in the input and pass to the network
    split_frac: Fraction of batches to keep in the training set
    
    
    Returns train_x, train_y, val_x, val_y
    """
    
    
    slice_size = batch_size * num_steps
    n_batches = int(len(chars) / slice_size)
    
    # Drop the last few characters to make only full batches
    x = chars[: n_batches*slice_size]
    y = chars[1: n_batches*slice_size + 1]
    
    # Split the data into batch_size slices, then stack them into a 2D matrix 
    x = np.stack(np.split(x, batch_size))
    y = np.stack(np.split(y, batch_size))
    
    # Now x and y are arrays with dimensions batch_size x n_batches*num_steps
    
    # Split into training and validation sets, keep the virst split_frac batches for training
    split_idx = int(n_batches*split_frac)
    train_x, train_y= x[:, :split_idx*num_steps], y[:, :split_idx*num_steps]
    val_x, val_y = x[:, split_idx*num_steps:], y[:, split_idx*num_steps:]
    
    return train_x, train_y, val_x, val_y

In [6]:
train_x, train_y, val_x, val_y = split_data(chars, 10, 200)

In [7]:
train_x.shape

(10, 178400)

In [8]:
train_x[:,:10]

array([[30, 55, 76, 12, 66, 65, 40, 11, 78,  8],
       [39, 25, 35, 11, 55, 65, 11, 14, 47, 26],
       [11,  4, 76, 66,  4, 55, 57, 25, 15, 11],
       [47, 66, 55, 65, 40, 11, 77, 47, 38, 21],
       [11, 66, 55, 65, 11, 21, 76, 25, 35, 48],
       [11, 82, 55, 40, 47, 38, 15, 55, 11, 21],
       [66, 11, 66, 47,  8, 35, 47, 23,  8,  8],
       [47, 11, 55, 65, 40, 46, 65, 21, 24,  2],
       [55, 76, 66, 11, 57, 46, 11, 66, 55, 65],
       [65, 40, 46, 65, 21, 24, 11, 76, 25, 35]])

I'll write another function to grab batches out of the arrays made by split data. Here each batch will be a sliding window on these arrays with size `batch_size X num_steps`. For example, if we want our network to train on a sequence of 100 characters, `num_steps = 100`. For the next batch, we'll shift this window the next sequence of `num_steps` characters. In this way we can feed batches to the network and the cell states will continue through on each batch.

In [9]:
def get_batch(arrs, num_steps):
    batch_size, slice_size = arrs[0].shape
    
    n_batches = int(slice_size/num_steps)
    for b in range(n_batches):
        yield [x[:, b*num_steps: (b+1)*num_steps] for x in arrs]

In [10]:
def build_rnn(num_classes, batch_size=50, num_steps=50, lstm_size=128, num_layers=2,
              learning_rate=0.001, grad_clip=5, sampling=False):
        
    if sampling == True:
        batch_size, num_steps = 1, 1

    tf.reset_default_graph()
    
    # Declare placeholders we'll feed into the graph
    with tf.name_scope('inputs'):
        inputs = tf.placeholder(tf.int32, [batch_size, num_steps], name='inputs')
        x_one_hot = tf.one_hot(inputs, num_classes, name='x_one_hot')
    
    with tf.name_scope('targets'):
        targets = tf.placeholder(tf.int32, [batch_size, num_steps], name='targets')
        y_one_hot = tf.one_hot(targets, num_classes, name='y_one_hot')
        y_reshaped = tf.reshape(y_one_hot, [-1, num_classes])
    
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    # Build the RNN layers
    with tf.name_scope("RNN_layers"):
        lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
        drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        cell = tf.contrib.rnn.MultiRNNCell([drop] * num_layers)
    
    with tf.name_scope("RNN_init_state"):
        initial_state = cell.zero_state(batch_size, tf.float32)

    # Run the data through the RNN layers
    with tf.name_scope("RNN_forward"):
        outputs, state = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=initial_state)
    
    final_state = state
    
    # Reshape output so it's a bunch of rows, one row for each cell output
    with tf.name_scope('sequence_reshape'):
        seq_output = tf.concat(outputs, axis=1,name='seq_output')
        output = tf.reshape(seq_output, [-1, lstm_size], name='graph_output')
    
    # Now connect the RNN putputs to a softmax layer and calculate the cost
    with tf.name_scope('logits'):
        softmax_w = tf.Variable(tf.truncated_normal((lstm_size, num_classes), stddev=0.1),
                               name='softmax_w')
        softmax_b = tf.Variable(tf.zeros(num_classes), name='softmax_b')
        logits = tf.matmul(output, softmax_w) + softmax_b

    with tf.name_scope('predictions'):
        preds = tf.nn.softmax(logits, name='predictions')
    
    
    with tf.name_scope('cost'):
        loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_reshaped, name='loss')
        cost = tf.reduce_mean(loss, name='cost')

    # Optimizer for training, using gradient clipping to control exploding gradients
    with tf.name_scope('train'):
        tvars = tf.trainable_variables()
        grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars), grad_clip)
        train_op = tf.train.AdamOptimizer(learning_rate)
        optimizer = train_op.apply_gradients(zip(grads, tvars))
    
    # Export the nodes 
    export_nodes = ['inputs', 'targets', 'initial_state', 'final_state',
                    'keep_prob', 'cost', 'preds', 'optimizer']
    Graph = namedtuple('Graph', export_nodes)
    local_dict = locals()
    graph = Graph(*[local_dict[each] for each in export_nodes])
    
    return graph

## Hyperparameters

Here I'm defining the hyperparameters for the network. The two you probably haven't seen before are `lstm_size` and `num_layers`. These set the number of hidden units in the LSTM layers and the number of LSTM layers, respectively. Of course, making these bigger will improve the network's performance but you'll have to watch out for overfitting. If your validation loss is much larger than the training loss, you're probably overfitting. Decrease the size of the network or decrease the dropout keep probability.

In [11]:
batch_size = 100
num_steps = 100
lstm_size = 512
num_layers = 2
learning_rate = 0.001

## Write out the graph for TensorBoard

In [20]:
model = build_rnn(len(vocab), 
                  batch_size=batch_size,
                  num_steps=num_steps,
                  learning_rate=learning_rate,
                  lstm_size=lstm_size,
                  num_layers=num_layers)

with tf.Session() as sess:
    
    sess.run(tf.global_variables_initializer())
    file_writer = tf.summary.FileWriter('./logs/3', sess.graph)

## Training

Time for training which is is pretty straightforward. Here I pass in some data, and get an LSTM state back. Then I pass that state back in to the network so the next batch can continue the state from the previous batch. And every so often (set by `save_every_n`) I calculate the validation loss and save a checkpoint.

In [14]:
!mkdir -p checkpoints\anna

In [19]:
epochs = 10
save_every_n = 200
train_x, train_y, val_x, val_y = split_data(chars, batch_size, num_steps)

model = build_rnn(len(vocab), 
                  batch_size=batch_size,
                  num_steps=num_steps,
                  learning_rate=learning_rate,
                  lstm_size=lstm_size,
                  num_layers=num_layers)

saver = tf.train.Saver(max_to_keep=100)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # Use the line below to load a checkpoint and resume training
    #saver.restore(sess, 'checkpoints/anna20.ckpt')
    
    n_batches = int(train_x.shape[1]/num_steps)
    iterations = n_batches * epochs
    for e in range(epochs):
        
        # Train network
        new_state = sess.run(model.initial_state)
        loss = 0
        for b, (x, y) in enumerate(get_batch([train_x, train_y], num_steps), 1):
            iteration = e*n_batches + b
            start = time.time()
            feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: 0.5,
                    model.initial_state: new_state}
            batch_loss, new_state, _ = sess.run([model.cost, model.final_state, model.optimizer], 
                                                 feed_dict=feed)
            loss += batch_loss
            end = time.time()
            print('Epoch {}/{} '.format(e+1, epochs),
                  'Iteration {}/{}'.format(iteration, iterations),
                  'Training loss: {:.4f}'.format(loss/b),
                  '{:.4f} sec/batch'.format((end-start)))
        
            
            if (iteration%save_every_n == 0) or (iteration == iterations):
                # Check performance, notice dropout has been set to 1
                val_loss = []
                new_state = sess.run(model.initial_state)
                for x, y in get_batch([val_x, val_y], num_steps):
                    feed = {model.inputs: x,
                            model.targets: y,
                            model.keep_prob: 1.,
                            model.initial_state: new_state}
                    batch_loss, new_state = sess.run([model.cost, model.final_state], feed_dict=feed)
                    val_loss.append(batch_loss)

                print('Validation loss:', np.mean(val_loss),
                      'Saving checkpoint!')
                saver.save(sess, "checkpoints\\anna\i{}_l{}_{:.3f}.ckpt".format(iteration, lstm_size, np.mean(val_loss)))

Epoch 1/10  Iteration 1/1780 Training loss: 4.4213 0.3213 sec/batch
Epoch 1/10  Iteration 2/1780 Training loss: 4.3814 0.3096 sec/batch
Epoch 1/10  Iteration 3/1780 Training loss: 4.2487 0.2907 sec/batch
Epoch 1/10  Iteration 4/1780 Training loss: 4.7591 0.2847 sec/batch
Epoch 1/10  Iteration 5/1780 Training loss: 4.6603 0.3000 sec/batch
Epoch 1/10  Iteration 6/1780 Training loss: 4.5125 0.3003 sec/batch
Epoch 1/10  Iteration 7/1780 Training loss: 4.3974 0.2971 sec/batch
Epoch 1/10  Iteration 8/1780 Training loss: 4.3026 0.2772 sec/batch
Epoch 1/10  Iteration 9/1780 Training loss: 4.2184 0.3012 sec/batch
Epoch 1/10  Iteration 10/1780 Training loss: 4.1431 0.3021 sec/batch
Epoch 1/10  Iteration 11/1780 Training loss: 4.0737 0.2824 sec/batch
Epoch 1/10  Iteration 12/1780 Training loss: 4.0135 0.3062 sec/batch
Epoch 1/10  Iteration 13/1780 Training loss: 3.9599 0.2957 sec/batch
Epoch 1/10  Iteration 14/1780 Training loss: 3.9136 0.2849 sec/batch
Epoch 1/10  Iteration 15/1780 Training loss

Epoch 1/10  Iteration 120/1780 Training loss: 3.2247 0.3214 sec/batch
Epoch 1/10  Iteration 121/1780 Training loss: 3.2231 0.3715 sec/batch
Epoch 1/10  Iteration 122/1780 Training loss: 3.2213 0.2948 sec/batch
Epoch 1/10  Iteration 123/1780 Training loss: 3.2193 0.3003 sec/batch
Epoch 1/10  Iteration 124/1780 Training loss: 3.2176 0.3004 sec/batch
Epoch 1/10  Iteration 125/1780 Training loss: 3.2156 0.2856 sec/batch
Epoch 1/10  Iteration 126/1780 Training loss: 3.2135 0.2901 sec/batch
Epoch 1/10  Iteration 127/1780 Training loss: 3.2115 0.2897 sec/batch
Epoch 1/10  Iteration 128/1780 Training loss: 3.2095 0.2884 sec/batch
Epoch 1/10  Iteration 129/1780 Training loss: 3.2074 0.2927 sec/batch
Epoch 1/10  Iteration 130/1780 Training loss: 3.2054 0.2932 sec/batch
Epoch 1/10  Iteration 131/1780 Training loss: 3.2034 0.2943 sec/batch
Epoch 1/10  Iteration 132/1780 Training loss: 3.2011 0.3008 sec/batch
Epoch 1/10  Iteration 133/1780 Training loss: 3.1988 0.2993 sec/batch
Epoch 1/10  Iteratio

Epoch 2/10  Iteration 237/1780 Training loss: 2.4414 0.3098 sec/batch
Epoch 2/10  Iteration 238/1780 Training loss: 2.4404 0.3314 sec/batch
Epoch 2/10  Iteration 239/1780 Training loss: 2.4402 0.2963 sec/batch
Epoch 2/10  Iteration 240/1780 Training loss: 2.4390 0.3209 sec/batch
Epoch 2/10  Iteration 241/1780 Training loss: 2.4380 0.2973 sec/batch
Epoch 2/10  Iteration 242/1780 Training loss: 2.4366 0.2998 sec/batch
Epoch 2/10  Iteration 243/1780 Training loss: 2.4352 0.3018 sec/batch
Epoch 2/10  Iteration 244/1780 Training loss: 2.4342 0.3003 sec/batch
Epoch 2/10  Iteration 245/1780 Training loss: 2.4330 0.2988 sec/batch
Epoch 2/10  Iteration 246/1780 Training loss: 2.4315 0.3033 sec/batch
Epoch 2/10  Iteration 247/1780 Training loss: 2.4298 0.3068 sec/batch
Epoch 2/10  Iteration 248/1780 Training loss: 2.4288 0.3074 sec/batch
Epoch 2/10  Iteration 249/1780 Training loss: 2.4276 0.3010 sec/batch
Epoch 2/10  Iteration 250/1780 Training loss: 2.4265 0.3013 sec/batch
Epoch 2/10  Iteratio

Epoch 2/10  Iteration 355/1780 Training loss: 2.3054 0.3016 sec/batch
Epoch 2/10  Iteration 356/1780 Training loss: 2.3044 0.2983 sec/batch
Epoch 3/10  Iteration 357/1780 Training loss: 2.1918 0.3309 sec/batch
Epoch 3/10  Iteration 358/1780 Training loss: 2.1465 0.3444 sec/batch
Epoch 3/10  Iteration 359/1780 Training loss: 2.1349 0.2979 sec/batch
Epoch 3/10  Iteration 360/1780 Training loss: 2.1285 0.3005 sec/batch
Epoch 3/10  Iteration 361/1780 Training loss: 2.1233 0.3028 sec/batch
Epoch 3/10  Iteration 362/1780 Training loss: 2.1165 0.3013 sec/batch
Epoch 3/10  Iteration 363/1780 Training loss: 2.1158 0.3014 sec/batch
Epoch 3/10  Iteration 364/1780 Training loss: 2.1169 0.3049 sec/batch
Epoch 3/10  Iteration 365/1780 Training loss: 2.1196 0.3005 sec/batch
Epoch 3/10  Iteration 366/1780 Training loss: 2.1186 0.2983 sec/batch
Epoch 3/10  Iteration 367/1780 Training loss: 2.1150 0.3539 sec/batch
Epoch 3/10  Iteration 368/1780 Training loss: 2.1127 0.3848 sec/batch
Epoch 3/10  Iteratio

Epoch 3/10  Iteration 472/1780 Training loss: 2.0355 0.2988 sec/batch
Epoch 3/10  Iteration 473/1780 Training loss: 2.0348 0.2971 sec/batch
Epoch 3/10  Iteration 474/1780 Training loss: 2.0340 0.2967 sec/batch
Epoch 3/10  Iteration 475/1780 Training loss: 2.0334 0.2978 sec/batch
Epoch 3/10  Iteration 476/1780 Training loss: 2.0327 0.3349 sec/batch
Epoch 3/10  Iteration 477/1780 Training loss: 2.0321 0.2998 sec/batch
Epoch 3/10  Iteration 478/1780 Training loss: 2.0313 0.3855 sec/batch
Epoch 3/10  Iteration 479/1780 Training loss: 2.0305 0.3144 sec/batch
Epoch 3/10  Iteration 480/1780 Training loss: 2.0299 0.2984 sec/batch
Epoch 3/10  Iteration 481/1780 Training loss: 2.0293 0.3039 sec/batch
Epoch 3/10  Iteration 482/1780 Training loss: 2.0284 0.3119 sec/batch
Epoch 3/10  Iteration 483/1780 Training loss: 2.0279 0.3068 sec/batch
Epoch 3/10  Iteration 484/1780 Training loss: 2.0273 0.2942 sec/batch
Epoch 3/10  Iteration 485/1780 Training loss: 2.0267 0.2977 sec/batch
Epoch 3/10  Iteratio

...

Epoch 8/10  Iteration 1289/1780 Training loss: 1.4711 0.2886 sec/batch
Epoch 8/10  Iteration 1290/1780 Training loss: 1.4703 0.3539 sec/batch
Epoch 8/10  Iteration 1291/1780 Training loss: 1.4703 0.2984 sec/batch
Epoch 8/10  Iteration 1292/1780 Training loss: 1.4693 0.3018 sec/batch
Epoch 8/10  Iteration 1293/1780 Training loss: 1.4689 0.2974 sec/batch
Epoch 8/10  Iteration 1294/1780 Training loss: 1.4683 0.2962 sec/batch
Epoch 8/10  Iteration 1295/1780 Training loss: 1.4678 0.2912 sec/batch
Epoch 8/10  Iteration 1296/1780 Training loss: 1.4681 0.3028 sec/batch
Epoch 8/10  Iteration 1297/1780 Training loss: 1.4673 0.3030 sec/batch
Epoch 8/10  Iteration 1298/1780 Training loss: 1.4680 0.2930 sec/batch
Epoch 8/10  Iteration 1299/1780 Training loss: 1.4677 0.2969 sec/batch
Epoch 8/10  Iteration 1300/1780 Training loss: 1.4677 0.2977 sec/batch
Epoch 8/10  Iteration 1301/1780 Training loss: 1.4675 0.3001 sec/batch
Epoch 8/10  Iteration 1302/1780 Training loss: 1.4674 0.2959 sec/batch
Epoch 

Epoch 8/10  Iteration 1404/1780 Training loss: 1.4510 0.2891 sec/batch
Epoch 8/10  Iteration 1405/1780 Training loss: 1.4507 0.2912 sec/batch
Epoch 8/10  Iteration 1406/1780 Training loss: 1.4507 0.2927 sec/batch
Epoch 8/10  Iteration 1407/1780 Training loss: 1.4508 0.2888 sec/batch
Epoch 8/10  Iteration 1408/1780 Training loss: 1.4508 0.2892 sec/batch
Epoch 8/10  Iteration 1409/1780 Training loss: 1.4507 0.2916 sec/batch
Epoch 8/10  Iteration 1410/1780 Training loss: 1.4506 0.3018 sec/batch
Epoch 8/10  Iteration 1411/1780 Training loss: 1.4505 0.3033 sec/batch
Epoch 8/10  Iteration 1412/1780 Training loss: 1.4503 0.3013 sec/batch
Epoch 8/10  Iteration 1413/1780 Training loss: 1.4504 0.2998 sec/batch
Epoch 8/10  Iteration 1414/1780 Training loss: 1.4507 0.2999 sec/batch
Epoch 8/10  Iteration 1415/1780 Training loss: 1.4506 0.3088 sec/batch
Epoch 8/10  Iteration 1416/1780 Training loss: 1.4505 0.2989 sec/batch
Epoch 8/10  Iteration 1417/1780 Training loss: 1.4503 0.2947 sec/batch
Epoch 

Epoch 9/10  Iteration 1520/1780 Training loss: 1.4132 0.3044 sec/batch
Epoch 9/10  Iteration 1521/1780 Training loss: 1.4132 0.3004 sec/batch
Epoch 9/10  Iteration 1522/1780 Training loss: 1.4127 0.3078 sec/batch
Epoch 9/10  Iteration 1523/1780 Training loss: 1.4122 0.3121 sec/batch
Epoch 9/10  Iteration 1524/1780 Training loss: 1.4117 0.3030 sec/batch
Epoch 9/10  Iteration 1525/1780 Training loss: 1.4117 0.2999 sec/batch
Epoch 9/10  Iteration 1526/1780 Training loss: 1.4115 0.3025 sec/batch
Epoch 9/10  Iteration 1527/1780 Training loss: 1.4113 0.3003 sec/batch
Epoch 9/10  Iteration 1528/1780 Training loss: 1.4111 0.3019 sec/batch
Epoch 9/10  Iteration 1529/1780 Training loss: 1.4108 0.2988 sec/batch
Epoch 9/10  Iteration 1530/1780 Training loss: 1.4106 0.3058 sec/batch
Epoch 9/10  Iteration 1531/1780 Training loss: 1.4104 0.2978 sec/batch
Epoch 9/10  Iteration 1532/1780 Training loss: 1.4102 0.2983 sec/batch
Epoch 9/10  Iteration 1533/1780 Training loss: 1.4099 0.2985 sec/batch
Epoch 

Epoch 10/10  Iteration 1635/1780 Training loss: 1.3851 0.3137 sec/batch
Epoch 10/10  Iteration 1636/1780 Training loss: 1.3851 0.2863 sec/batch
Epoch 10/10  Iteration 1637/1780 Training loss: 1.3851 0.2984 sec/batch
Epoch 10/10  Iteration 1638/1780 Training loss: 1.3847 0.3009 sec/batch
Epoch 10/10  Iteration 1639/1780 Training loss: 1.3841 0.2995 sec/batch
Epoch 10/10  Iteration 1640/1780 Training loss: 1.3830 0.2996 sec/batch
Epoch 10/10  Iteration 1641/1780 Training loss: 1.3816 0.3005 sec/batch
Epoch 10/10  Iteration 1642/1780 Training loss: 1.3809 0.3050 sec/batch
Epoch 10/10  Iteration 1643/1780 Training loss: 1.3802 0.3053 sec/batch
Epoch 10/10  Iteration 1644/1780 Training loss: 1.3807 0.2988 sec/batch
Epoch 10/10  Iteration 1645/1780 Training loss: 1.3802 0.2896 sec/batch
Epoch 10/10  Iteration 1646/1780 Training loss: 1.3794 0.3002 sec/batch
Epoch 10/10  Iteration 1647/1780 Training loss: 1.3794 0.3002 sec/batch
Epoch 10/10  Iteration 1648/1780 Training loss: 1.3784 0.3000 se

Epoch 10/10  Iteration 1749/1780 Training loss: 1.3651 0.2973 sec/batch
Epoch 10/10  Iteration 1750/1780 Training loss: 1.3652 0.2980 sec/batch
Epoch 10/10  Iteration 1751/1780 Training loss: 1.3652 0.2993 sec/batch
Epoch 10/10  Iteration 1752/1780 Training loss: 1.3651 0.2990 sec/batch
Epoch 10/10  Iteration 1753/1780 Training loss: 1.3648 0.3093 sec/batch
Epoch 10/10  Iteration 1754/1780 Training loss: 1.3646 0.3004 sec/batch
Epoch 10/10  Iteration 1755/1780 Training loss: 1.3647 0.3023 sec/batch
Epoch 10/10  Iteration 1756/1780 Training loss: 1.3646 0.3073 sec/batch
Epoch 10/10  Iteration 1757/1780 Training loss: 1.3645 0.3073 sec/batch
Epoch 10/10  Iteration 1758/1780 Training loss: 1.3645 0.3023 sec/batch
Epoch 10/10  Iteration 1759/1780 Training loss: 1.3645 0.3078 sec/batch
Epoch 10/10  Iteration 1760/1780 Training loss: 1.3645 0.3109 sec/batch
Epoch 10/10  Iteration 1761/1780 Training loss: 1.3642 0.3023 sec/batch
Epoch 10/10  Iteration 1762/1780 Training loss: 1.3642 0.3049 se

In [ ]:
tf.train.get_checkpoint_state('checkpoints\anna')

## Sampling

Now that the network is trained, we'll can use it to generate new text. The idea is that we pass in a character, then the network will predict the next character. We can use the new one, to predict the next one. And we keep doing this to generate all new text. I also included some functionality to prime the network with some text by passing in a string and building up a state from that.

The network gives us predictions for each character. To reduce noise and make things a little less random, I'm going to only choose a new character from the top N most likely characters.



In [21]:
def pick_top_n(preds, vocab_size, top_n=5):
    p = np.squeeze(preds)
    p[np.argsort(p)[:-top_n]] = 0
    p = p / np.sum(p)
    c = np.random.choice(vocab_size, 1, p=p)[0]
    return c

In [22]:
def sample(checkpoint, n_samples, lstm_size, vocab_size, prime="The "):
    prime = "Far"
    samples = [c for c in prime]
    model = build_rnn(vocab_size, lstm_size=lstm_size, sampling=True)
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, checkpoint)
        new_state = sess.run(model.initial_state)
        for c in prime:
            x = np.zeros((1, 1))
            x[0,0] = vocab_to_int[c]
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.preds, model.final_state], 
                                         feed_dict=feed)

        c = pick_top_n(preds, len(vocab))
        samples.append(int_to_vocab[c])

        for i in range(n_samples):
            x[0,0] = c
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.preds, model.final_state], 
                                         feed_dict=feed)

            c = pick_top_n(preds, len(vocab))
            samples.append(int_to_vocab[c])
        
    return ''.join(samples)

In [24]:
checkpoint = "checkpoints/anna/i10_l512_3.188.ckpt"
samp = sample(checkpoint, 2000, lstm_size, len(vocab), prime="Far")
print(samp)

Fare et   iei  eeeieit et  n e e tt  n  e et t it   it ene      t   et ti e ni tneee   ien e  i et t  nn  eett  ttt  nnt           e et  ini      eit t i nt  nt  ieeni t     e ne n        e t  eii teeeen    in  etit t   eei tt    i tn  netntte  et t e i   ni t i e t   ineei    n   n     e nt te    i t  e  nt nn e  ii  te   n i   ne  ei  t tt t ii  ettti   nntnn tin  t  i  nt n e ieitentt i  eini i t   tt  e teinnent  n e n       t tne  ii  ttn tt     it   e   t    ei eie ie  e e     t et  i e      t nt  nt nnn    ten et   tetitet enn t  tei       e     i  e  i   i  it iet  n n  een  e ei neien  einet t   t i  t  nt ee  e   n ni   ee   n     n n  itt   n e     e tt t    neen e e n ee n e  ee etnt  t n  e it ntte    e e e iit innn  n   e et tn t  tte     t nte ee e     ne t  i  nnte neeie  teti en  tni eie n    te ie n     n  nit nn  n     en nenteee  e  n enttte n  t n n  e  te   e     itee      n    it   nin  iee it  tinn ee  e  niei   ee  nii ni      tn t ein i    t   e  e  e  t ie tt

In [ ]:
checkpoint = "checkpoints/anna/i200_l512_2.432.ckpt"
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

In [ ]:
checkpoint = "checkpoints/anna/i600_l512_1.750.ckpt"
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

In [ ]:
checkpoint = "checkpoints/anna/i1000_l512_1.484.ckpt"
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)